In [1]:
import pandas as pd

from bs4 import BeautifulSoup

from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords

In [2]:
from gensim import models
import numpy as np

In [3]:
import gensim
from gensim import corpora
from pprint import pprint
import string

In [6]:
df_chunk = pd.read_csv(r"D:\data\openparliament\statements_nospeaker_en_small.csv", chunksize=1000)

In [7]:
content_en_list = []  # append the content from each chunk df here 

In [8]:
stop_words = set(stopwords.words('english'))

In [54]:
punctuation = '!"#$%&()*+,-./:;<=>@[]^_`{|}~' + "''"
pun_trans = str.maketrans(dict.fromkeys(punctuation, ' '))

In [9]:
n_trans = str.maketrans(dict.fromkeys('\\n',' '))

In [10]:
print("here")

here


In [ ]:
%%time
df_chunk = pd.read_csv(r"D:\data\openparliament\statements_nospeaker_en.csv", chunksize=10000)

content_en_list = []  # append the content from each chunk df here 

def tokenize_column (row):
    soup = BeautifulSoup(row)
    text = soup.text.lower()
    #text = text.translate(pun_trans)
    #text = text.translate(n_trans)
    #text = text.replace('\\n', ' ')
    words = word_tokenize(text)
    return words

stop_words = set(stopwords.words('english'))

def remove_stopwords(word_tokens):
    words = [w for w in word_tokens if not w in stop_words]
    #remove digits:
    return words


# Each chunk is in df format
for chunk in df_chunk:  
    # perform data filtering 
    chunk_filter = chunk.drop(['Unnamed: 0'], axis=1)
    content_en = chunk.content_en
    #print(type(content_en))
    content_en = content_en.apply(tokenize_column)
    #content_en = content_en.apply(remove_stopwords)
    chunk_filter['tokens'] = content_en
    #print(type(chunk_filter))
    #print('done')
    # Once the data filtering is done, append the chunk to list
    content_en_list.append(chunk_filter)

In [11]:
dff = content_en_list[0]

In [12]:
dff.head()

,id,politician_id,riding_id,party_id,end_date,start_date,id.1,document_id,time,h1_en,...,urlcache,h1_fr,h2_fr,h3_fr,who_fr,who_context_fr,wordcount_en,tokenized_content,slug_length,tokens
0,2611,3465,35066,4,2005-11-29,1997-09-22,232373,1878,2001-05-03 13:50:00-04,Government Orders,...,/debates/2001/5/3/marlene-catterall-1/,NaN,NaN,NaN,NaN,NaN,\N,"['mr.', 'speaker', ',', 'i', 'rise', 'on', 'a'...",3,"[mr., speaker, ,, i, rise, on, a, point, of, o..."
1,3210,3465,70224,4,1997-04-27,1994-01-17,232373,1878,2001-05-03 13:50:00-04,Government Orders,...,/debates/2001/5/3/marlene-catterall-1/,NaN,NaN,NaN,NaN,NaN,\N,"['mr.', 'speaker', ',', 'i', 'rise', 'on', 'a'...",3,"[mr., speaker, ,, i, rise, on, a, point, of, o..."
2,4305,173,70358,1,\N,2015-10-19,645329,388,2008-02-14 13:15:00-05,Routine Proceedings,...,/debates/2008/2/14/tom-lukiwski-7/,NaN,NaN,NaN,NaN,NaN,\N,"['finally', ',', 'mr.', 'speaker', ',', 'i', '...",3,"[finally, ,, mr., speaker, ,, i, ask, that, al..."
3,1534,173,47007,1,2015-10-19,2004-10-04,645329,388,2008-02-14 13:15:00-05,Routine Proceedings,...,/debates/2008/2/14/tom-lukiwski-7/,NaN,NaN,NaN,NaN,NaN,\N,"['finally', ',', 'mr.', 'speaker', ',', 'i', '...",3,"[finally, ,, mr., speaker, ,, i, ask, that, al..."
4,1541,43,47014,1,2015-10-19,2004-10-04,313253,1621,2004-02-06 10:05:00-05,NaN,...,/debates/2004/2/6/garry-breitkreuz-1/,NaN,NaN,NaN,NaN,NaN,\N,"['mr.', 'speaker', ',', 'i', 'rise', 'today', ...",3,"[mr., speaker, ,, i, rise, today, on, a, quest..."


In [13]:
len(content_en_list)

5

In [14]:
%%time
df_concat = pd.concat(content_en_list)

Wall time: 42 ms


In [15]:
df_concat.head()

,id,politician_id,riding_id,party_id,end_date,start_date,id.1,document_id,time,h1_en,...,urlcache,h1_fr,h2_fr,h3_fr,who_fr,who_context_fr,wordcount_en,tokenized_content,slug_length,tokens
0,2611,3465,35066,4,2005-11-29,1997-09-22,232373,1878,2001-05-03 13:50:00-04,Government Orders,...,/debates/2001/5/3/marlene-catterall-1/,NaN,NaN,NaN,NaN,NaN,\N,"['mr.', 'speaker', ',', 'i', 'rise', 'on', 'a'...",3,"[mr., speaker, ,, i, rise, on, a, point, of, o..."
1,3210,3465,70224,4,1997-04-27,1994-01-17,232373,1878,2001-05-03 13:50:00-04,Government Orders,...,/debates/2001/5/3/marlene-catterall-1/,NaN,NaN,NaN,NaN,NaN,\N,"['mr.', 'speaker', ',', 'i', 'rise', 'on', 'a'...",3,"[mr., speaker, ,, i, rise, on, a, point, of, o..."
2,4305,173,70358,1,\N,2015-10-19,645329,388,2008-02-14 13:15:00-05,Routine Proceedings,...,/debates/2008/2/14/tom-lukiwski-7/,NaN,NaN,NaN,NaN,NaN,\N,"['finally', ',', 'mr.', 'speaker', ',', 'i', '...",3,"[finally, ,, mr., speaker, ,, i, ask, that, al..."
3,1534,173,47007,1,2015-10-19,2004-10-04,645329,388,2008-02-14 13:15:00-05,Routine Proceedings,...,/debates/2008/2/14/tom-lukiwski-7/,NaN,NaN,NaN,NaN,NaN,\N,"['finally', ',', 'mr.', 'speaker', ',', 'i', '...",3,"[finally, ,, mr., speaker, ,, i, ask, that, al..."
4,1541,43,47014,1,2015-10-19,2004-10-04,313253,1621,2004-02-06 10:05:00-05,NaN,...,/debates/2004/2/6/garry-breitkreuz-1/,NaN,NaN,NaN,NaN,NaN,\N,"['mr.', 'speaker', ',', 'i', 'rise', 'today', ...",3,"[mr., speaker, ,, i, rise, today, on, a, quest..."


In [17]:
df_concat.columns

Index(['id', 'politician_id', 'riding_id', 'party_id', 'end_date',
       'start_date', 'id.1', 'document_id', 'time', 'h1_en', 'h2_en',
       'member_id', 'who_en', 'content_en', 'sequence_en', 'wordcount',
       'politician_id.1', 'procedural', 'h3_en', 'who_hocid', 'content_fr',
       'statement_type', 'written_question', 'source_id', 'who_context_en',
       'slug', 'urlcache', 'h1_fr', 'h2_fr', 'h3_fr', 'who_fr',
       'who_context_fr', 'wordcount_en', 'tokenized_content', 'slug_length',
       'tokens'],
      dtype='object')

In [23]:
politicians = list(set(df_concat.politician_id))

In [24]:
len(politicians)

1051

In [25]:
vocab_count_dict = dict()

In [26]:
vocab_count_dict = vocab_count_dict.fromkeys(politicians, set())

In [28]:
politicians[56]

4156

In [29]:
vocab_count_dict[4156]

{'syriac',
 '90b',
 'pshcp',
 'later—but',
 'examples—',
 'predetermi',
 'pied',
 'recoi',
 'although',
 'mobilized—a',
 'devitt',
 'doggo',
 'sloa',
 'reciprocal',
 'coped',
 'alpho',
 'greate',
 'cerology',
 'jayewarde',
 'orexic',
 'more—to',
 'tagi',
 'appraisers',
 'advisers—a',
 'flask',
 'carbert',
 'gaga',
 'facts',
 '7008',
 'correctly—is',
 'slagged',
 'zuke',
 'crête—a',
 'k0k',
 '1910s',
 'sparky',
 'hypercaffei',
 'petito',
 'project—how',
 'calls—dr',
 'wabash',
 'fpp',
 'quiry—that',
 'commmittee',
 'prescheduled',
 'boots—or',
 'diabolically',
 'brochet',
 'ufacturers',
 '2012—we',
 'ows—my',
 'hv',
 'disaggregated',
 'abbassi',
 'haap',
 'horticulturalist',
 'ziadeh',
 'columbia—',
 'macdo',
 'cipl',
 'weatherva',
 'royalty—royalty',
 'carso',
 'er—from',
 'beji',
 'typist',
 'kershaw',
 'eumeces',
 'dustries—with',
 '—pulli',
 'osteria',
 'objected',
 'bourget',
 'ow—heaped',
 'ordermade',
 'folds',
 'isheed',
 'sulted—u',
 'radioday',
 '4666',
 'zmapp',
 'chhaya',
 '

note: filter out '\\n'

In [30]:
s = 'court.\\nthen'

In [31]:
'\\n' in s

True

In [32]:
s.split('\\n')

['court.', 'then']

In [33]:
len(vocab_count_dict[4156])

125226

In [34]:
p_set = vocab_count_dict[4156]

In [35]:
type(p_set)

set

In [36]:
p_set.remove('court.\\nthen')

KeyError: 'court.\\nthen'

In [24]:
df = df_concat[df_concat.politician_id == 4156]

In [25]:
df.shape

(123, 36)

In [26]:
statements = df_concat.content_en.values

In [28]:
len(statements)

50000

In [34]:
count = 0
dash = []
for state in statements:
    if '—' in state:
        count+=1
        #print(state)
        dash.append(state)

In [35]:
count

7935

In [39]:
'-' == '-'

True

In [55]:
soup = BeautifulSoup(dash[1])
text = soup.text.lower()
text = text.translate(pun_trans)

In [56]:
text = text.replace('\\n', ' ')
text = text.replace('—', ' ')

print(text)

mr  speaker  this morning we are beginning the last hour of debate at second reading of bill c 420  an act to amend the food and drugs act  put forward by our hon  colleague from the canadian alliance  let us remember that in 1997 the house of commons standing committee on health was given a mandate to study all aspects of the issue of legislating natural health products  the committee was charged with holding consultations  making analyses and formulating recommendations concerning the legislative and regulatory regime governing traditional medicines  homeopathic preparations  and vitamin and mineral supplements  the committee was also expected to consult broadly with stakeholders  including associations  individuals representing consumers  manufacturers  distributors  growers  importers  exporters and retailers  we see that this bill  like the committee s mandate  has reached a great many individuals who were very concerned about the regulation of natural health products  the committ

In [57]:
tokens = list(set(word_tokenize(text)))

In [58]:
'—'=='-'

False

In [59]:
tokens = [t for t in tokens if not t.isdigit()]

In [61]:
from nltk.stem import PorterStemmer 

ps = PorterStemmer() 

In [63]:
tokens = [ps.stem(w) for w in tokens]

In [64]:
tokens

['regul',
 'or',
 'ye',
 'for',
 'job',
 'specialist',
 'refer',
 'uniqu',
 'know',
 'broadli',
 'own',
 'conclus',
 'benefit',
 'bloc',
 'unfortun',
 'than',
 'concern',
 'appear',
 'health',
 'to',
 'lead',
 'conclus',
 'indic',
 'mani',
 'expect',
 'be',
 'ha',
 'sinc',
 'skill',
 'level',
 'present',
 'gener',
 'cultur',
 'regulatoryapproach',
 'state',
 'defin',
 'exclud',
 'formul',
 'like',
 'either',
 'receiv',
 'bodi',
 'ask',
 'read',
 'we',
 'repres',
 'recommend',
 'again',
 'that',
 'previou',
 'show',
 'sometim',
 'product',
 'categori',
 'repres',
 'soon',
 'oppos',
 'canadian',
 'strictli',
 'not',
 'same',
 'industri',
 'befor',
 'new',
 'did',
 'quebecoi',
 'which',
 'taken',
 'also',
 'reach',
 'ignor',
 'amend',
 'it',
 'current',
 'therefor',
 'studi',
 'pharmaceut',
 'safeguard',
 'includ',
 'kept',
 'take',
 'more',
 'lost',
 'export',
 'treat',
 'impact',
 'thu',
 'charg',
 'addit',
 'these',
 'hear',
 'creat',
 'go',
 'need',
 'possibl',
 'unaccept',
 'under',


note: getting rid of doubled words like "regulatoryapproaches" needs more thought - it's not frequent, so leaving it isn't a massive problem